# Queens Problem

### importem llibreries que utilitzarem i definim variables globals


In [220]:
import numpy as np
import random

INITIAL_POPULATION = 10
BOARD_SIZE = 8
MUTATION_CHANCE = 0.1
MIN_POPULAITON_MUTATION=0.1

### Definir Agent, població i display

In [221]:
def init_agent(board_size):
    return random.sample(range(board_size), board_size)

def display_agent(agent):
    for queenPosition in agent:
        result = []

        for el in range(len(agent)):
            result.append(1 if el == queenPosition else 0)

        print(result)

def init_population(initial_population_size, board_size):
    return [init_agent(board_size) for _ in range(initial_population_size)]


_population = init_population(INITIAL_POPULATION, BOARD_SIZE)
_agent = _population[0]
assert BOARD_SIZE == len(_agent)
assert INITIAL_POPULATION == len(_population)
display_agent(_agent)
# assert BOARD_SIZE*BOARD_SIZE == len(display_agent(_agent))

[0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]


### definir crossover

In [222]:
def crossover(parent1, parent2):
    board_size = len(parent1)

    crossover_point = random.randint(1, board_size - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]

    # Remove duplicates
    unique_values = set(child)
    all_values = set(range(board_size))
    missing_values = list(all_values - unique_values)

    for i in range(board_size):
        if child.count(child[i]) > 1:
            child[i] = missing_values.pop(0)
    
    return child

child = crossover(_population[0], _population[1])
print(_population[0])
print(_population[1])
print(child)

[4, 0, 6, 1, 3, 2, 5, 7]
[5, 3, 0, 1, 4, 6, 2, 7]
[4, 0, 6, 1, 3, 2, 5, 7]


### definir mutació


In [223]:
def mutation(agent):
    pos1, pos2 = random.sample(range(len(agent)), 2)
    agent[pos1], agent[pos2] = agent[pos2], agent[pos1]
    return agent

def mutation_with_chance(agent, mutation_chance):
    if random.random() < mutation_chance:
        return mutation(agent)

    return agent

_agentCopy = _agent.copy()
print(mutation_with_chance(_agentCopy, 0.1))
print(_agent)

[4, 0, 6, 1, 3, 2, 5, 7]
[4, 0, 6, 1, 3, 2, 5, 7]


### definir crossover de tota la població amb mutació

In [224]:
def replicate(population, mutation_chance, min_population_mutation):
    new_population = []
    population_size = len(population)

    # Generate childs
    for _ in range(population_size // 2):
        parent1, parent2 = random.sample(population, 2)
        
        child1 = crossover(parent1, parent2)
        child2 = crossover(parent2, parent1)
        
        child1 = mutation_with_chance(child1, mutation_chance)
        child2 = mutation_with_chance(child2, mutation_chance)
        
        new_population.append(child1)
        new_population.append(child2)

    # Generate one more child if is necesary
    if population_size % 2 == 1:
        parent1, parent2 = random.sample(population, 2)
        child = crossover(parent1, parent2)
        child = mutation_with_chance(child, mutation_chance)
        new_population.append(child)

    # Merge population
    combined_population = population + new_population

    # Get minimum percentage of mutated agents
    total_population_size = len(combined_population)
    required_mutations = max(1, int(min_population_mutation * total_population_size))
    
    # Get mutated agents in combined population
    mutated_agents = [agent for agent in combined_population if agent != mutation_with_chance(agent, 0)]
    num_mutated = len(mutated_agents)

    # Add percentage of mutated agents if is necesary
    while num_mutated < required_mutations:
        # Mutate an agent non mutated
        non_mutated_agents = [agent for agent in combined_population if agent not in mutated_agents]
        if non_mutated_agents:
            agent_to_mutate = random.choice(non_mutated_agents)
            mutated_agent = mutation(agent_to_mutate)
            mutated_agents.append(mutated_agent)
            combined_population[combined_population.index(agent_to_mutate)] = mutated_agent
            num_mutated += 1

    # Return same size that population input
    return random.sample(combined_population, population_size)

replicate(_population, MUTATION_CHANCE, MIN_POPULAITON_MUTATION)


[[5, 2, 0, 6, 7, 4, 1, 3],
 [4, 0, 6, 1, 3, 2, 5, 7],
 [3, 1, 6, 5, 7, 4, 0, 2],
 [5, 3, 0, 1, 4, 6, 2, 7],
 [0, 4, 6, 1, 3, 2, 5, 7],
 [4, 5, 0, 6, 2, 7, 1, 3],
 [1, 3, 6, 2, 7, 4, 0, 5],
 [4, 7, 3, 6, 1, 5, 0, 2],
 [5, 3, 0, 1, 4, 6, 2, 7],
 [3, 1, 6, 2, 7, 4, 0, 5]]

### definir fitness function

mirar diagonals i treure repetides

In [225]:
def fitness_agent(agent):
    conflicts = 0
    for i in range(len(agent)):
        for j in range(i + 1, len(agent)):
            if abs(agent[i] - agent[j]) == j - i:
                conflicts += 1
    return conflicts

def fitness_population(population):
    return [fitness_agent(agent) for agent in population]

display_agent(_agent)
print(fitness_agent(_agent))
assert fitness_agent(_agent) <= BOARD_SIZE

[0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
6


### Population Selection (by score)


In [226]:
def sort_population(population, fitness_list):
    paired = list(zip(population, fitness_list))
    sorted_paired = sorted(paired, key=lambda x: x[1], reverse=False)
    return [x[0] for x in sorted_paired]

def select_population(population, fitness_list, population_size):
    sorted_population = sort_population(population, fitness_list)
    return sorted_population[:population_size]

print(select_population(_population, fitness_population(_population), BOARD_SIZE))
print(_population)

[[4, 7, 3, 6, 1, 5, 0, 2], [3, 1, 6, 2, 7, 4, 0, 5], [5, 2, 0, 6, 7, 4, 1, 3], [4, 5, 0, 6, 2, 7, 1, 3], [5, 3, 0, 1, 4, 6, 2, 7], [7, 3, 6, 1, 5, 4, 2, 0], [2, 5, 0, 6, 4, 3, 7, 1], [5, 4, 1, 6, 3, 0, 2, 7]]
[[4, 0, 6, 1, 3, 2, 5, 7], [5, 3, 0, 1, 4, 6, 2, 7], [4, 5, 0, 6, 2, 7, 1, 3], [4, 7, 3, 6, 1, 5, 0, 2], [1, 6, 4, 3, 5, 7, 0, 2], [3, 1, 6, 2, 7, 4, 0, 5], [7, 3, 6, 1, 5, 4, 2, 0], [2, 5, 0, 6, 4, 3, 7, 1], [5, 2, 0, 6, 7, 4, 1, 3], [5, 4, 1, 6, 3, 0, 2, 7]]


### definir main

(seguir els pasos de classe)

In [227]:
def main(population_size, board_size, mutation_chance, min_mutation_childs):
    population = init_population(population_size, board_size)
    generation = 0

    while True:
        fitness_list = fitness_population(population)
        best_fitness = min(fitness_list)
        best_agent = population[fitness_list.index(best_fitness)]

        if generation % 100 == 0:
            print(f"Population {generation}: {best_fitness}")

        if best_fitness == 0:
            print(f"Solución óptima encontrada en la generación {generation}: {best_agent}")
            break

        selected_population = select_population(population, fitness_list, population_size)
        new_population = replicate(selected_population, mutation_chance, min_mutation_childs)
        population = new_population
        generation += 1

    return best_agent

### Provar el codi

In [264]:
POPULATION_SIZE = 300
BOARD_SIZE = 12
MUTATION_CHANCE = 0.1
MIN_POPULAITON_MUTATION=0

agent = main(POPULATION_SIZE, BOARD_SIZE, MUTATION_CHANCE, MIN_POPULAITON_MUTATION)
display_agent(agent)

Population 0: 2
Population 100: 2
Population 200: 2
Population 300: 3
Population 400: 2
Population 500: 2
Population 600: 2
Population 700: 2
Population 800: 2
Population 900: 3
Population 1000: 3
Population 1100: 2
Population 1200: 3
Population 1300: 3
Population 1400: 3
Population 1500: 3
Population 1600: 2
Population 1700: 2
Population 1800: 2
Population 1900: 2
Population 2000: 2
Solución óptima encontrada en la generación 2059: [8, 3, 11, 4, 1, 5, 0, 10, 7, 2, 6, 9]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
